[View in Colaboratory](https://colab.research.google.com/github/abunickabhi/IRproject/blob/master/Word_embedding.ipynb)

In [42]:
!git clone https://github.com/abunickabhi/IRproject.git ; rm -rf embed/ ;mkdir embed ;cd IRproject/ ; git pull origin
!pip3 install Cython
!pip3 install word2vec
!pip3 install fasttext
!pip3 install gensim
!pip3 install torch
!pip3 install PyDictionary


fatal: destination path 'IRproject' already exists and is not an empty directory.
remote: Counting objects: 4, done.
remote: Compressing objects: 100% (4/4), done.
remote: Total 4 (delta 1), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (4/4), done.
From https://github.com/abunickabhi/IRproject
   58bea75..0744332  master     -> origin/master
Updating 58bea75..0744332
Fast-forward
 med_eng.vec | 333 ++++++++++++++++++++++++++++++++++++++++++++++++++
 med_tml.vec | 400 ++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
 2 files changed, 733 insertions(+)
 create mode 100644 med_eng.vec
 create mode 100644 med_tml.vec


**Importing all the required libraries**

In [0]:
import gensim
import word2vec
import fasttext
import numpy as np
import torch
import torch.nn as nn
from torch.autograd import Variable
from torch.nn import functional as F
from PyDictionary import PyDictionary
from torch import optim




**Loading and Processing the source words i.e. English**



In [43]:
max_vocab = 10000
emb_dim = 300
src_lang = 'eng'

src_word2id = {}
src_embeddings = []
a= 'IRproject/med_eng.vec'

with open(a) as f:
    for i,line in enumerate(f):
        if i==0:
            split = line.split()
            assert len(split) == 2
            assert emb_dim == int(split[1])
        else:
            word, vect = line.rstrip().split(' ', 1)
            vect = np.fromstring(vect, sep=' ')
            if np.linalg.norm(vect)==0: #to avoid null embeddings
                vect[0] = 0.01
            assert word not in src_word2id
            assert vect.shape == (emb_dim, )
            src_word2id[word] = len(src_word2id)
            src_embeddings.append(vect[None,:])
        if i > max_vocab:
            break
            
src_id2word = {v: k for k,v in src_word2id.items()}
src_embeddings = np.concatenate(src_embeddings,0)
src_embeddings = torch.from_numpy(src_embeddings).float()
src_emb = nn.Embedding(len(src_word2id), emb_dim, sparse=True)
src_emb.weight.data.copy_(src_embeddings)


 0.1073  0.0089  0.0006  ...   0.0050  0.1173 -0.0400
 0.0897  0.0160 -0.0571  ...   0.1559 -0.0254 -0.0259
 0.0004  0.0032 -0.0204  ...   0.2070  0.0689 -0.0467
          ...             ⋱             ...          
-0.0988 -0.0694  0.0330  ...   0.1917  0.0759 -0.0402
 0.1017 -0.0606 -0.0056  ...   0.1346  0.1418 -0.0015
 0.0682  0.0611 -0.0201  ...  -0.1313  0.0914  0.0037
[torch.FloatTensor of size 332x300]

**Loading the source language i.e. Tamil into a Dictionary**

In [44]:
src_dico = PyDictionary(src_id2word, src_word2id, src_lang)

tgt_lang = 'ta'
tgt_word2id = {}
tgt_embeddings = []
b= 'IRproject/med_tml.vec'

with open(b) as f:
    for i,line in enumerate(f):
        if i==0:
            split = line.split()
            assert len(split) == 2
            assert emb_dim == int(split[1])
        else:
            word, vect = line.rstrip().split(' ', 1)
            vect = np.fromstring(vect, sep=' ')
            if np.linalg.norm(vect)==0: #to avoid null embeddings
                vect[0] = 0.01
            assert word not in tgt_word2id
            assert vect.shape == (emb_dim, )
            tgt_word2id[word] = len(tgt_word2id)
            tgt_embeddings.append(vect[None,:])
        if i > max_vocab:
            break

tgt_id2word = {v:k for k,v in tgt_word2id.items()}

tgt_dico = PyDictionary(tgt_id2word, tgt_word2id, tgt_lang)
tgt_embeddings = np.concatenate(tgt_embeddings,0)
tgt_embeddings = torch.from_numpy(tgt_embeddings).float()
tgt_emb = nn.Embedding(len(tgt_word2id), emb_dim, sparse=True)
tgt_emb.weight.data.copy_(tgt_embeddings)





-0.0051 -0.0092  0.0438  ...  -0.0510  0.0875  0.0016
 0.0472 -0.0408  0.0252  ...   0.1013  0.0901  0.0136
-0.0163  0.0031  0.0754  ...  -0.0167  0.1091 -0.0434
          ...             ⋱             ...          
 0.0262 -0.0360 -0.0021  ...  -0.0091  0.0209 -0.0222
-0.0092 -0.0611 -0.0357  ...  -0.0284 -0.0225 -0.0451
-0.0156 -0.0003  0.0217  ...  -0.0131  0.0056  0.0035
[torch.FloatTensor of size 399x300]

In [45]:
mapping = nn.Linear(emb_dim, emb_dim, bias=False)
mapping.weight.data.copy_(torch.diag(torch.ones(emb_dim)))



    1     0     0  ...      0     0     0
    0     1     0  ...      0     0     0
    0     0     1  ...      0     0     0
       ...          ⋱          ...       
    0     0     0  ...      1     0     0
    0     0     0  ...      0     1     0
    0     0     0  ...      0     0     1
[torch.FloatTensor of size 300x300]

**Create a self class in Discriminator class and define the parameters of training**

In [62]:
disc_layers = 2
disc_dim_hidden = 2048
disc_dropout = 0
disc_inp_dropout = 0.1

class Discriminator(nn.Module):
    def __init__(self):
        super(Discriminator, self).__init__()
        
        self.emb_dim = emb_dim
        self.disc_layers = disc_layers
        self.disc_dim_hidden = disc_dim_hidden
        self.disc_dropout = disc_dropout
        self.disc_inp_dropout = disc_inp_dropout
        
        layers = [nn.Dropout(self.disc_inp_dropout)]
        for i in range(self.disc_layers + 1):
            input_dim = self.emb_dim if i == 0 else self.disc_dim_hidden
            output_dim = 1 if i==self.disc_layers else self.disc_dim_hidden
            layers.append(nn.Linear(input_dim, output_dim))
            if i < self.disc_layers:
                layers.append(nn.LeakyReLU(0.2))
                layers.append(nn.Dropout(self.disc_dropout))
        layers.append(nn.Sigmoid())
        self.layers = nn.Sequential(*layers)
        
    def forward(self, x):
        return self.layers(x).view(-1)
      
discriminator = Discriminator()

src_emb.cuda()
tgt_emb.cuda()
mapping.cuda()
discriminator.cuda()


Discriminator(
  (layers): Sequential(
    (0): Dropout(p=0.1)
    (1): Linear(in_features=300, out_features=2048, bias=True)
    (2): LeakyReLU(0.2)
    (3): Dropout(p=0)
    (4): Linear(in_features=2048, out_features=2048, bias=True)
    (5): LeakyReLU(0.2)
    (6): Dropout(p=0)
    (7): Linear(in_features=2048, out_features=1, bias=True)
    (8): Sigmoid()
  )
)

**Use Stochastic Gradient Optimizer from Torch optim library to get source and target prediction and accuracy score**

****

In [0]:
optim_fn = optim.SGD
optim_params = {'lr': 0.1}

map_optimizer = optim_fn(mapping.parameters(), **optim_params)
disc_optimizer = optim_fn(discriminator.layers.parameters(), **optim_params)
epoch_size = 1000
batch_size = 32
disc_steps = 5
disc_most_freq = 750
disc_smooth = 0.1
map_beta = 0.001

def get_disc_xy(volatile):
    """
    Get discriminator input batch / output target.
    """
    # select random word IDs
    bs = batch_size
    mf = disc_most_freq
    assert mf <= min(len(src_dico), len(tgt_dico))
    src_ids = torch.FloatTensor(bs).random_(mf)
    tgt_ids = torch.FloatTensor(bs).random_(mf)
    #src_ids = src_ids.cuda()
    #tgt_ids = tgt_ids.cuda()

    # get word embeddings
    src_emb = src_emb(Variable(src_ids, volatile=True))
    tgt_emb = tgt_emb(Variable(tgt_ids, volatile=True))
    src_emb = mapping(Variable(src_emb.data, volatile=volatile))
    tgt_emb = Variable(tgt_emb.data, volatile=volatile)

    # input / target
    x = torch.cat([src_emb, tgt_emb], 0)
    y = torch.FloatTensor(2 * bs).zero_()
    y[:bs] = 1 - disc_smooth
    y[bs:] = dis_smooth
    y = Variable(y.cuda())

    return x, y

def eval_dis(self, to_log):
  
    bs = 128
    src_preds = []
    tgt_preds = []

    discriminator.eval()

    for i in range(0, src_emb.num_embeddings, bs):
        emb = Variable(src_emb.weight[i:i + bs].data, volatile=True)
        preds = discriminator(mapping(emb))
        src_preds.extend(preds.data.cpu().tolist())

    for i in range(0,tgt_emb.num_embeddings, bs):
        emb = Variable(tgt_emb.weight[i:i + bs].data, volatile=True)
        preds = discriminator(emb)
        tgt_preds.extend(preds.data.cpu().tolist())

    src_pred = np.mean(src_preds)
    tgt_pred = np.mean(tgt_preds)
    print("Discriminator source / target predictions: %.5f / %.5f"
                % (src_pred, tgt_pred))

    src_accu = np.mean([x >= 0.5 for x in src_preds])
    tgt_accu = np.mean([x < 0.5 for x in tgt_preds])
    dis_accu = ((src_accu * src_emb.num_embeddings + tgt_accu * tgt_emb.num_embeddings) /
                    (src_emb.num_embeddings + tgt_emb.num_embeddings))
    print("Discriminator source / target / global accuracy: %.5f / %.5f / %.5f"
                    % (src_accu, tgt_accu, dis_accu))
   

In [63]:
 print(src_emb)
 print(tgt_emb)


 0.0645  0.0053  0.0004  ...   0.0030  0.0705 -0.0240
 0.0620  0.0111 -0.0395  ...   0.1078 -0.0176 -0.0179
 0.0003  0.0020 -0.0130  ...   0.1314  0.0437 -0.0296
          ...             ⋱             ...          
-0.0553 -0.0389  0.0185  ...   0.1073  0.0425 -0.0225
 0.0567 -0.0338 -0.0031  ...   0.0750  0.0790 -0.0008
 0.0381  0.0341 -0.0112  ...  -0.0733  0.0510  0.0021
[torch.cuda.FloatTensor of size 332x300 (GPU 0)]


-0.0033 -0.0060  0.0286  ...  -0.0333  0.0571  0.0010
 0.0343 -0.0297  0.0183  ...   0.0736  0.0655  0.0099
-0.0066  0.0013  0.0304  ...  -0.0067  0.0440 -0.0175
          ...             ⋱             ...          
 0.0438 -0.0602 -0.0035  ...  -0.0152  0.0350 -0.0371
-0.0089 -0.0592 -0.0346  ...  -0.0275 -0.0218 -0.0437
-0.0285 -0.0005  0.0397  ...  -0.0240  0.0102  0.0064
[torch.cuda.FloatTensor of size 399x300 (GPU 0)]



**Adversarial Training**

In [0]:
def adv_training(epochs):
    for epoch in range(epochs):
        n_words_proc = 0
        for n_iter in range(0,epoch_size, batch_size):
            for _ in range(disc_steps): # Discriminator training
                discriminator.train() # sets the module in training mode ex adds dropout and batchnorm
                x,y = get_disc_xy(volatile=True)
                preds = discriminator(Variable(x.data))
                loss = F.binary_cross_entropy(preds,y)
                disc_optimizer.zero_grad()
                loss.backward()
                disc_optimizer.step()
                
            
            #Mapping step
            discriminator.eval() # Puts the module in evaluation mode.
            x, y = get_disc_xy(volatile=False)
            preds = discriminator(Variable(x.data))
            loss = F.binary_cross_entropy(preds, 1-y)
            map_optimizer.zero_grad()
            loss.backward()
            map_optmizer.step()
            orthogonalize()
            n_words_proc += 2 * batch_size
            
        # embeddings / discriminator evaluation
        eval_dis()


exp_path = ""
def export_embeddings(src_emb, tgt_emb, params):
    
    src_id2word = src_dico.id2word
    tgt_id2word = tgt_dico.id2word
    n_src = len(src_id2word)
    n_tgt = len(tgt_id2word)
    dim = src_emb.shape[1]
    src_path = os.path.join(exp_path, 'vectors-%s.txt' % src_lang)
    tgt_path = os.path.join(exp_path, 'vectors-%s.txt' % tgt_lang)
    # source embeddings
    with open(src_path, 'w') as f:
        f.write("%i %i\n" % (n_src, dim))
        for i in range(len(src_id2word)):
            f.write("%s %s\n" % (src_id2word[i], " ".join(str(x) for x in src_emb[i])))
   # target embeddings
    with open(tgt_path, 'w') as f:
        f.write("%i %i\n" % (n_tgt, dim))
        for i in range(len(tgt_id2word)):
            f.write("%s %s\n" % (tgt_id2word[i], " ".join(str(x) for x in tgt_emb[i])))
            


In [71]:
print(src_id2word)
print(tgt_id2word)

{0: ',', 1: 'the', 2: '.', 3: 'and', 4: 'of', 5: 'to', 6: 'in', 7: 'a', 8: '"', 9: ':', 10: ')', 11: 'that', 12: '(', 13: 'is', 14: 'for', 15: 'on', 16: '*', 17: 'with', 18: 'as', 19: 'it', 20: 'The', 21: 'or', 22: 'was', 23: "'", 24: "'s", 25: 'by', 26: 'from', 27: 'at', 28: 'I', 29: 'this', 30: 'you', 31: '/', 32: 'are', 33: '=', 34: 'not', 35: '-', 36: 'have', 37: '?', 38: 'be', 39: 'which', 40: ';', 41: 'all', 42: 'his', 43: 'has', 44: 'one', 45: 'their', 46: 'about', 47: 'but', 48: 'an', 49: '|', 50: 'said', 51: 'more', 52: 'page', 53: 'he', 54: 'your', 55: 'will', 56: 'its', 57: 'so', 58: 'were', 59: 'had', 60: 'also', 61: 'only', 62: 'if', 63: 'time', 64: 'some', 65: 'people', 66: 'like', 67: 'who', 68: 'them', 69: 'other', 70: 'they', 71: 'when', 72: 'Wikipedia', 73: 'article', 74: 'what', 75: '#', 76: 'just', 77: '!', 78: 'any', 79: 'after', 80: 'there', 81: 'would', 82: 'can', 83: 'In', 84: 'her', 85: 'talk', 86: 'use', 87: 'then', 88: 'into', 89: 'up', 90: '...', 91: 'we', 9